# 【日本語】EDA of March Machine Learning Mania 2022 Men’s

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# **データロード**

In [ ]:
PATH = '../input/mens-march-mania-2022/'

cities = pd.read_csv(PATH + 'MDataFiles_Stage1/Cities.csv')
conferences = pd.read_csv(PATH + 'MDataFiles_Stage1/Conferences.csv')
teams = pd.read_csv(PATH + 'MDataFiles_Stage1/MTeams.csv')
regular_compact_result = pd.read_csv(PATH + 'MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
regular_detail_result = pd.read_csv(PATH + 'MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
conference_tourney = pd.read_csv(PATH + 'MDataFiles_Stage1/MConferenceTourneyGames.csv')
seasons = pd.read_csv(PATH + 'MDataFiles_Stage1/MSeasons.csv')
ncaa_compact_result = pd.read_csv(PATH + 'MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
ncaa_detail_result = pd.read_csv(PATH + 'MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
ncaa_seed_roundslot = pd.read_csv(PATH + 'MDataFiles_Stage1/MNCAATourneySeedRoundSlots.csv')
ncaa_seed = pd.read_csv(PATH + 'MDataFiles_Stage1/MNCAATourneySeeds.csv')
ncaa_slot = pd.read_csv(PATH + 'MDataFiles_Stage1/MNCAATourneySlots.csv')
secondary_compact_result = pd.read_csv(PATH + 'MDataFiles_Stage1/MSecondaryTourneyCompactResults.csv')
secondary_teams = pd.read_csv(PATH + 'MDataFiles_Stage1/MSecondaryTourneyTeams.csv')
coaches = pd.read_csv(PATH + 'MDataFiles_Stage1/MTeamCoaches.csv')
team_conferences = pd.read_csv(PATH + 'MDataFiles_Stage1/MTeamConferences.csv')
team_spellings = pd.read_csv(PATH + 'MDataFiles_Stage1/MTeamSpellings.csv',encoding='shift-jis')
game_cities = pd.read_csv(PATH + 'MDataFiles_Stage1/MGameCities.csv')
massey_ordinals = pd.read_csv(PATH + 'MDataFiles_Stage1/MMasseyOrdinals.csv')
submission = pd.read_csv(PATH + 'MDataFiles_Stage1/MSampleSubmissionStage1.csv')

# **キーワード**
ディビジョン ・・・　リーグ。3つ存在。当コンペはディビジョン1（D1）が対象  
カンファレンス ・・・ ディビジョンに紐づくリーグ  
レギュラーシーズン ・・・ カンファレンス内外での試合  
カンファレンストーナメント　・・・　各カンファレンスにてNCAAトーナメント出場校を決める大会。優勝32校がNACCトーナメントへ  
  
NCAAトーナメント　・・・　全米大学No.1を決める大会（出場校68校（カンファレンストーナメント優勝32校+選抜38校)）  
ブロック（地区） ・・・ トーナメントはランキングに基づき、4つのブロックに分けられる  
シード ・・・　各ブロックにて第1〜16シードまで割り振られ、その順位に基づき対戦カードが決められる  
ブレーインゲーム ・・・ トーナメントの0回戦。選抜で選ばれた8チームが対戦し、勝った4チーム（ファーストフォー）がトーナメント1回戦へ進む  
  
非NCAAトーナメント　・・・　NCAAトーナメントに出場できなかったチームが参加する大会

# **コンペ概要**
- 2022年のNCAAトーナメント（ディビジョン1）における試合の勝敗予測  
- 出場チームの全組み合わせ(2,278試合）の勝敗予測をし、実際に行われた試合について予測スコアを競う（※対象試合は全試合なのか限定的なのか不確かです。ご注意ください※）  
- Stage1（テスト（モデル構築））、Stage2（本番）にわかれる。Stage1の参加は必須ではない   
- 評価指標は LogLoss  
## Stage1
- 過去5年間にNCAAトーナメントに参加したチームの全組み合わせ勝敗予測（2,278試合×5年）  
- 推論データ（組み合わせ記載）は、コンペ開始当初から公開  
## Stage2
- 2022年のNCAAトーナメントに参加するチームの全組み合わせ試合勝敗予測（2,278試合） 
- 推論データ（組み合わせ記載）は、3/14（月）に公開
- 推論データ締切: 3/17（木）PM3 UTC  
- トーナメントは3/15〜4/4にかけて開催（4/4以降にコンペの順位が確定すると思われます）


# **<a id="提出データ">提出データ</a>**
- season_team1(ID)_team2(ID)の形式で提示される  
- team1がteam2に勝つ確率をpredカラムへ入力

In [ ]:
submission.head()

# **テーブル概要**
### マスタ  
[Cities.csv](#Cities) ・・・ 都市の情報  
[Conferences.csv](#Conferences) ・・・ カンファレンスの情報  
[MTeams.csv](#Teams)　・・・　チームの情報  
[MTeamSpellings.csv](#TeamSpellings) ・・・ チーム名の綴り 
  
### レギュラーシーズン
[MRegularSeasonCompactResults.csv](#RegularCompactResult) ・・・　レギュラーシーズンの結果概要  
[MRegularSeasonDetailedResults.csv](#RegularDetailResult) ・・・　レギュラーシーズンの結果詳細  
  
### カンファレンストーナメント 
[MTeamConferences.csv](#TeamConferences) ・・・　各チームのカンファレンス所属情報  
[MConferenceTourneyGames.csv](#ConferenceTourney)　・・・ カンファレンストーナメントでの結果概要  
  
### NCAAトーナメント
[MSeasons.csv](#Seasons) ・・・　各シーズンのトーナメントブロックに関する情報  
[MNCAATourneyCompactResults.csv](#NCAACompactResult) ・・・　NCAAトーナメントの結果概要  
[MNCAATourneyDetailedResults.csv](#NCAADetailResult) ・・・　NCAAトーナメントの結果詳細  
[MNCAATourneySeedRoundSlots.csv](#NCAASeedRoundSlot)  ・・・    
[MNCAATourneySeeds.csv](#NCAASeed) ・・・ シードチームに関する情報  
[MNCAATourneySlots.csv](#NCAASlot) ・・・　シードのペアリングメカニズムに関する情報  
  
### 非NCAAトーナメント
[MSecondaryTourneyCompactResults.csv](#SecondaryCompactResult) ・・・ 非NACCトーナメントの結果概要  
[MSecondaryTourneyTeams.csv](#SecondaryTeams) ・・・ 非NACCトーナメントの参加チーム情報  
  
### その他 
[MTeamCoaches.csv](#Coaches) ・・・　監督の情報  
[MGameCities.csv](#GameCities) ・・・　全試合の結果概要  
[MMasseyOrdinals.csv](#MasseyOrdinals) ・・・　チームのランキング  
  
### 提出データ
[MSampleSubmissionStage1.csv](#提出データ) ・・・　提出データサンプル

# **EDA**

## マスタ

### <a id="Cities">Cities</a>
都市の情報
- CityID: 都市のID  
- City: 都市名　　
- State: 州の頭文字

In [ ]:
cities.head()

In [ ]:
print(f'レコード数: {len(cities)}')

### <a id="Conferences">Conferences</a>
カンファレンスの情報
- ConfAbbrev: カンファレンス略称  
- Description: カンファレンス名

In [ ]:
conferences.head()

In [ ]:
print(f'レコード数: {len(conferences)}')

### <a id="Teams">Teams</a>
チームの情報
- TeamID: チームID
- TeamName: チーム名
- FirstD1Season: データセット中で大学がディビジョン1であった最初のシーズン
- LastD1Season: データセット中で大学がディビジョン1であった最後のシーズン

In [ ]:
teams.head()

In [ ]:
print(f'レコード数: {len(teams)}')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True)

ax1.set_title('FirstD1Season')
ax1.hist(teams.FirstD1Season)
ax2.set_title('LastD1Season')
ax2.hist(teams.LastD1Season)

plt.tight_layout()
plt.show()

### <a id="TeamSpellings">TeamSpellings</a>
チーム名の綴り
- TeamNameSpelling: チーム名の綴り
- TeamID: チームID

In [ ]:
team_spellings.head()

In [ ]:
print(f'チーム数: {team_spellings.TeamID.nunique()}')
print(f'綴り数: {team_spellings.TeamNameSpelling.nunique()}')

## レギュラーシーズン

### <a id="RegularCompactResult">RegularCompactResult</a>
レギュラーシーズンの結果概要
- Season: 年
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）
- WTeamID: 勝チームのID
- WScore: 勝チームのスコア
- LTeamID: 敗チームのID
- LScore: 敗チームのスコア
- WLoc: 試合場所。勝チームのホームであった場合'H', アウェイの場合'A', いずれでもない場合'N'
- NumOT: 延長戦の回数

In [ ]:
regular_compact_result.head()

In [ ]:
print(f'レコード数: {len(regular_compact_result)}')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True)

ax1.set_title('WScore')
ax1.hist(regular_compact_result.WScore)
ax2.set_title('LScore')
ax2.hist(regular_compact_result.LScore)

plt.tight_layout()
plt.show()

print(f'平均Wscore: {sum(regular_compact_result.WScore)/len(regular_compact_result)}')
print(f'平均Lscore: {sum(regular_compact_result.LScore)/len(regular_compact_result)}')

### <a id="RegularDetailResult">RegularDetailResult</a>
レギュラーシーズンの結果詳細
- Season: 年 
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）  
- WTeamID: 勝チームのID
- WScore: 勝チームのスコア
- LTeamID: 敗チームのID
- LScore: 敗チームのスコア
- WLoc: 試合場所。勝チームのホームであった場合'H', アウェイの場合'A', いずれでもない場合'N'
- NumOT: 延長戦の回数
- WFGM: 勝チームのシュート成功本数  
- WFGA: 勝チームのシュート試投本数  
- WFGM3: 勝チームの3ポイントシュート成功本数  
- WFGA3: 勝チームの3ポイントシュート試投本数  
- WFTM: 勝チームのフリースロー成功本数  
- WFTA: 勝チームのフリースロー試投本数  
- WOR: 勝チームのオフェンスリバウンド取得回数  
- WDR: 勝チームのディフェンスリバウンド取得回数  
- WAst: 勝チームのアシスト回数  
- WTO: 勝チームのターンオーバー回数  
- WStl: 勝チームのスティール回数  
- WBlk: 勝チームのシュートブロック回数  
- WPF: 勝チームのパーソナルファール回数  
- LFGM: 敗チームのシュート成功本数 
- LFGA: 敗チームのシュート試投本数  
- LFGM3: 敗チームの3ポイントシュート成功本数  
- LFGA3: 敗チームの3ポイントシュート試投本数  
- LFTM: 敗チームのフリースロー成功本数  
- LFTA: 敗チームのフリースロー試投本数  
- LOR: 敗チームのオフェンスリバウンド取得回数  
- LDR: 敗チームのディフェンスリバウンド取得回数  
- LAst: 敗チームのアシスト回数  
- LTO: 敗チームのターンオーバー回数  
- LStl: 敗チームのスティール回数  
- LBlk: 敗チームのシュートブロック回数  
- LPF: 敗チームのパーソナルファール回数  

In [ ]:
regular_detail_result.head()

In [ ]:
print(f'レコード数: {len(regular_detail_result)}')

cols = ['Season', 'DayNum', 'WTeamID', 'LTeamID']
tmp = regular_detail_result.drop(cols, axis=1)
fig = plt.figure(figsize=(20, 30))
for i, col in enumerate(tmp.columns):
    plt.subplot2grid((11, 3), (i//3, i%3))
    plt.hist(tmp[col])
    plt.title(col)

## カンファレンストーナメント

### <a id="TeamConferences">TeamConferences</a>
各チームのカンファレンス所属情報
- Season: 年  
- TeamID: チームID  
- ConfAbbrev: カンファレンス略称

In [ ]:
team_conferences.head()

In [ ]:
print(f'チーム数: {team_conferences.TeamID.nunique()}')
print(f'カンファレンス数: {team_conferences.ConfAbbrev.nunique()}')

### <a id="ConferenceTourney">ConferenceTourney</a>
カンファレンストーナメントでの結果概要
- Season: 年
- ConfAbbrev: カンファレンス略称
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）  
- WTeamID: 勝チームのID
- LTeamID: 負チームのID

In [ ]:
conference_tourney.head()

In [ ]:
print(f'レコード数: {len(conference_tourney)}')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True)
tmp = conference_tourney.groupby('WTeamID', as_index=False).agg({'Season': pd.Series.count})
ax1.set_title('Num of Win')
ax1.hist(tmp.Season)
tmp = conference_tourney.groupby('LTeamID', as_index=False).agg({'Season': pd.Series.count})
ax2.set_title('Num of Lose')
ax2.hist(tmp.Season)

plt.tight_layout()
plt.show()

## NCAAトーナメント

### <a id="Seasons">Seasons</a>
各シーズンのトーナメントブロックに関する情報
- Season: 年
- DayZero: そのシーズンのDayNum=0に対応する日付（NCAAトーナメントの決勝がDayNum=154となるように調整されている）
- RegionW, RegionX, RegionY, RegionZ: トーナメントブロック名（頭文字のアルファベット順に配列）

In [ ]:
seasons.head()

###  <a id="NCAACompactResult">NCAACompactResult</a>
NCAAトーナメントの結果概要
- Season: 年
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）
- WTeamID: 勝チームのID
- WScore: 勝チームのスコア
- LTeamID: 敗チームのID
- LScore: 敗チームのスコア
- WLoc: 試合場所。勝チームのホームであった場合'H', アウェイの場合'A', いずれでもない場合'N'
- NumOT: 延長戦の回数

In [ ]:
ncaa_compact_result.head()

In [ ]:
print(f'レコード数: {len(ncaa_compact_result)}')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True)

ax1.set_title('WScore')
ax1.hist(ncaa_compact_result.WScore)
ax2.set_title('LScore')
ax2.hist(ncaa_compact_result.LScore)

plt.tight_layout()
plt.show()

print(f'平均Wscore: {sum(ncaa_compact_result.WScore)/len(ncaa_compact_result)}')
print(f'平均Lscore: {sum(ncaa_compact_result.LScore)/len(ncaa_compact_result)}')

### <a id="NCAADetailResult">NCAADetailResult</a>
NCAAトーナメントの結果詳細
- Season: 年 
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）  
- WTeamID: 勝チームのID
- WScore: 勝チームのスコア
- LTeamID: 敗チームのID
- LScore: 敗チームのスコア
- WLoc: 試合場所。勝チームのホームであった場合'H', アウェイの場合'A', いずれでもない場合'N'
- NumOT: 延長戦の回数
- WFGM: 勝チームのシュート成功本数  
- WFGA: 勝チームのシュート試投本数  
- WFGM3: 勝チームの3ポイントシュート成功本数  
- WFGA3: 勝チームの3ポイントシュート試投本数  
- WFTM: 勝チームのフリースロー成功本数  
- WFTA: 勝チームのフリースロー試投本数  
- WOR: 勝チームのオフェンスリバウンド取得回数  
- WDR: 勝チームのディフェンスリバウンド取得回数  
- WAst: 勝チームのアシスト回数  
- WTO: 勝チームのターンオーバー回数  
- WStl: 勝チームのスティール回数  
- WBlk: 勝チームのシュートブロック回数  
- WPF: 勝チームのパーソナルファール回数  
- LFGM: 敗チームのシュート成功本数 
- LFGA: 敗チームのシュート試投本数  
- LFGM3: 敗チームの3ポイントシュート成功本数  
- LFGA3: 敗チームの3ポイントシュート試投本数  
- LFTM: 敗チームのフリースロー成功本数  
- LFTA: 敗チームのフリースロー試投本数  
- LOR: 敗チームのオフェンスリバウンド取得回数  
- LDR: 敗チームのディフェンスリバウンド取得回数  
- LAst: 敗チームのアシスト回数  
- LTO: 敗チームのターンオーバー回数  
- LStl: 敗チームのスティール回数  
- LBlk: 敗チームのシュートブロック回数  
- LPF: 敗チームのパーソナルファール回数  

In [ ]:
ncaa_detail_result.head()

In [ ]:
print(f'レコード数: {len(ncaa_detail_result)}')

cols = ['Season', 'DayNum', 'WTeamID', 'LTeamID']
tmp = ncaa_detail_result.drop(cols, axis=1)
fig = plt.figure(figsize=(20, 30))
for i, col in enumerate(tmp.columns):
    plt.subplot2grid((11, 3), (i//3, i%3))
    plt.hist(tmp[col])
    plt.title(col)

### <a id="NCAASeedRoundSlot">NCAASeedRoundSlot</a>
- Seed: シード番号。頭文字が所属ブロック（W、X、Y、Z)、続く2文字がそのブロック内での通し番号
- GameRound: ラウンド。0:プレイインゲーム、ラウンド1,2:1週目の試合、ラウンド3,4:2週目の試合。ラウンド5,6:準決、決勝  
- GameSlot: 'MNCAATourneySlots'ファイルで定義
- EarlyDayNum: 可能性として最も早く試合が行われる日付（値は'MSeasns.csv'ファイルの'DayZero'(日付)カラムからのオフセット）  
- LateDayNum: 可能性として最も遅く試合が行われる日付（値は'MSeasns.csv'ファイルの'DayZero'(日付)カラムからのオフセット）

In [ ]:
ncaa_seed_roundslot.head()

In [ ]:
print(f'レコード数: {len(ncaa_seed_roundslot)}')

### <a id="NCAASeed">NCAASeed</a>
シードチームに関する情報
- Season: 年  
- Seed: シードチームの識別子。頭文字が所属ブロック（W、X、Y、Z)、続く2文字がそのブロック内でのシード番号(01〜16)※プレーインチームの場合、シード番号の後にa,bが付属する
- TeamID: チームID

In [ ]:
ncaa_seed.head()

In [ ]:
print(f'チーム数: {ncaa_seed.TeamID.nunique()}')
print(f'シード数: {ncaa_seed.Seed.nunique()}')

tmp = ncaa_seed.groupby('TeamID', as_index=False).agg({'Season': pd.Series.count})
plt.title('Num of Seed of each team')
plt.hist(tmp.Season)

plt.tight_layout()
plt.show()

### <a id="NCAASlot">NCAASlot</a>
シードのペアリングメカニズムに関する情報
- Season: 年  
- Slot: 試合の識別子。最初の2文字がラウンド、続く2文字がシード番号
- StrongSeed: この試合で強いと予想されたチームのシード番号  
- WeakSeed: この試合で弱いと予想されたチームのシード番号

In [ ]:
ncaa_slot.head()

In [ ]:
print(f'レコード数: {len(ncaa_slot)}')

## 非NCAAトーナメント

### <a id="SecondaryCompactResult">SecondaryCompactResult</a>
非NACCトーナメントの結果概要
- Season: 年
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）
- WTeamID: 勝チームのID
- WScore: 勝チームのスコア
- LTeamID: 敗チームのID
- LScore: 敗チームのスコア
- WLoc: 試合場所。勝チームのホームであった場合'H', アウェイの場合'A', いずれでもない場合'N'
- NumOT: 延長戦の回数
- SecondaryTourney: トーナメント名（NIT、CBI、CIT、V16）

In [ ]:
secondary_compact_result.head()

In [ ]:
print(f'レコード数: {len(secondary_compact_result)}')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True)

ax1.set_title('WScore')
ax1.hist(secondary_compact_result.WScore)
ax2.set_title('LScore')
ax2.hist(secondary_compact_result.LScore)

plt.tight_layout()
plt.show()

print(f'平均Wscore: {sum(secondary_compact_result.WScore)/len(secondary_compact_result)}')
print(f'平均Lscore: {sum(secondary_compact_result.LScore)/len(secondary_compact_result)}')

### <a id="SecondaryTeams">SecondaryTeams</a>
非NACCトーナメントの参加チーム情報
- Season: 年  
- SecondaryTourney: トーナメント名（NIT、CBI、CIT、V16）
- TeamID: チームID

In [ ]:
secondary_teams.head()

In [ ]:
print(f'チーム数: {secondary_teams.TeamID.nunique()}')

tmp = secondary_teams.groupby('SecondaryTourney', as_index=False).agg({'Season': pd.Series.count})
plt.title('Num of each SecondaryTourney')
plt.bar(tmp.SecondaryTourney,tmp.Season)

plt.show()

## その他

### <a id="Coaches">Coaches</a>
監督の情報
- Season: 年  
- TeamID: チームID  
- FirstDayNum: そのシーズンの最初の日（値はMSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット） ※0ならシーズン初めから活動
- LastDayNum: そのシーズンの最後の日（値はMSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット） ※154ならシーズン最後まで活動
- CoachName: コーチ名

In [ ]:
coaches.head()

In [ ]:
print(f'コーチ数: {coaches.CoachName.nunique()}')

### <a id="GameCities">GameCities</a>
全試合の結果概要
- Season: 年
- DayNum: 日付（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）
- WTeamID: 勝チームID
- LTeamID: 負チームID
- CRType: 試合が行われたシーズン、トーナメント
- CityID: 都市ID

In [ ]:
game_cities.head()

In [ ]:
print(f'都市数: {game_cities.CityID.nunique()}')

tmp = game_cities.groupby('CityID', as_index=False).agg({'Season': pd.Series.count})
plt.title('Num of games of each City')
plt.hist(tmp.Season)
# plt.boxplot(tmp.Season)

plt.show()

### <a id="MasseyOrdinals">MasseyOrdinals</a>
チームのランキング
- Season: 年  
- RankingDayNum: ランキング発表日（MSeasns.csvファイルの'DayZero'(日付)カラムからのオフセット）※0〜133となる
- SystemName: ランキング選考システム
- TeamID: チームID
- OrdinalRank: ランキング

In [ ]:
massey_ordinals.head()

In [ ]:
tmp = massey_ordinals.groupby(['TeamID'], as_index=False
                             ).agg({'OrdinalRank': 'mean'}).set_axis(['TeamID','mean_rank'], axis=1)
print('ランキング平均TOP10チーム')
tmp.sort_values('mean_rank').head(10)